# Queries Module - Fresh Data Queries

Refresh live data in Module 3. Market prices use MODULE_1_INPUT data.

```python
%run queries_module.ipynb

df_stocks = get_current_stocks()
df_prices = get_current_prices()
df_wac = get_current_wac()
df_cart = get_current_cart_rules()
```


In [ ]:
# =============================================================================
# IMPORTS & SNOWFLAKE CONNECTION
# =============================================================================
import pandas as pd
import snowflake.connector
import os
import sys

# Add parent directory to path to import setup_environment_2
sys.path.append('..')
import setup_environment_2

# Initialize environment variables (loads Snowflake credentials)
setup_environment_2.initialize_env()

def query_snowflake(query):
    """Execute a query on Snowflake and return results as DataFrame."""
    con = snowflake.connector.connect(
        user=os.environ["SNOWFLAKE_USERNAME"],
        account=os.environ["SNOWFLAKE_ACCOUNT"],
        password=os.environ["SNOWFLAKE_PASSWORD"],
        database=os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        data = cur.fetchall()
        columns = [desc[0].lower() for desc in cur.description]
        return pd.DataFrame(data, columns=columns)
    finally:
        con.close()

def get_snowflake_timezone():
    result = query_snowflake("SHOW PARAMETERS LIKE 'TIMEZONE'")
    return result.value[0] if len(result) > 0 else "UTC"

TIMEZONE = get_snowflake_timezone()
print(f"Queries Module | Timezone: {TIMEZONE}")


In [ ]:
# =============================================================================
# QUERY 1: CURRENT STOCKS (from data_extraction.ipynb)
# =============================================================================
STOCK_QUERY = '''
SELECT 
    pw.warehouse_id,
    pw.product_id,
    pw.available_stock::INTEGER AS stocks
FROM product_warehouse pw
WHERE pw.warehouse_id NOT IN (6, 9, 10)
    AND pw.is_basic_unit = 1
'''

def get_current_stocks():
    """Get fresh stock levels."""
    print("Fetching current stocks...")
    df = query_snowflake(STOCK_QUERY)
    print(f"  Loaded {len(df)} records")
    return df


In [ ]:
# =============================================================================
# QUERY 2: CURRENT PRICES (from data_extraction.ipynb)
# =============================================================================
CURRENT_PRICES_QUERY = f'''
WITH skus_prices AS (
    WITH local_prices AS (
        SELECT  
            CASE 
                WHEN cpu.cohort_id IN (700, 695) THEN 'Cairo'
                WHEN cpu.cohort_id IN (701) THEN 'Giza'
                WHEN cpu.cohort_id IN (704, 698) THEN 'Delta East'
                WHEN cpu.cohort_id IN (703, 697) THEN 'Delta West'
                WHEN cpu.cohort_id IN (696, 1123, 1124, 1125, 1126) THEN 'Upper Egypt'
                WHEN cpu.cohort_id IN (702, 699) THEN 'Alexandria'
            END AS region,
            cohort_id,
            pu.product_id,
            pu.packing_unit_id,
            pu.basic_unit_count,
            AVG(cpu.price) AS price
        FROM cohort_product_packing_units cpu
        JOIN PACKING_UNIT_PRODUCTS pu ON pu.id = cpu.product_packing_unit_id
        WHERE cpu.cohort_id IN (700,701,702,703,704,695,696,697,698,699,1123,1124,1125,1126)
            AND cpu.created_at::date <> '2023-07-31'
            AND cpu.is_customized = TRUE
        GROUP BY ALL
    ),
    
    live_prices AS (
        SELECT 
            region, cohort_id, product_id, 
            pu_id AS packing_unit_id, 
            buc AS basic_unit_count, 
            NEW_PRICE AS price
        FROM materialized_views.DBDP_PRICES
        WHERE created_at = CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date
            AND DATE_PART('hour', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::time) 
                BETWEEN SPLIT_PART(time_slot, '-', 1)::int AND (SPLIT_PART(time_slot, '-', 1)::int) + 1
            AND cohort_id IN (700,701,702,703,704,695,696,697,698,699,1123,1124,1125,1126)
    ),
    
    prices AS (
        SELECT *
        FROM (
            SELECT *, 1 AS priority FROM live_prices
            UNION ALL
            SELECT *, 2 AS priority FROM local_prices
        )
        QUALIFY ROW_NUMBER() OVER (PARTITION BY region, cohort_id, product_id, packing_unit_id ORDER BY priority) = 1
    )
    
    SELECT region, cohort_id, product_id, price
    FROM prices
    WHERE basic_unit_count = 1
        AND ((product_id = 1309 AND packing_unit_id = 2) OR (product_id <> 1309))
)

SELECT distinct region, cohort_id, product_id, price as current_price
FROM skus_prices
'''

def get_current_prices():
    """Get fresh current prices."""
    print("Fetching current prices...")
    df = query_snowflake(CURRENT_PRICES_QUERY)
    print(f"  Loaded {len(df)} records")
    return df


In [ ]:
# =============================================================================
# QUERY 3: CURRENT WAC (from data_extraction.ipynb)
# =============================================================================
WAC_QUERY = f'''
SELECT 
    product_id,
    wac_p
FROM finance.all_cogs
WHERE CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) 
    BETWEEN from_date AND to_date
'''

def get_current_wac():
    """Get fresh WAC (Weighted Average Cost)."""
    print("Fetching current WAC...")
    df = query_snowflake(WAC_QUERY)
    print(f"  Loaded {len(df)} records")
    return df


In [ ]:
# =============================================================================
# QUERY 4: CURRENT CART RULES (from data_extraction.ipynb)
# =============================================================================
COHORT_IDS = [700, 701, 702, 703, 704, 1123, 1124, 1125, 1126]

CART_RULES_QUERY = f'''
SELECT 
    cppu.cohort_id,
    pup.product_id,
    pup.basic_unit_count,
    COALESCE(cppu.MAX_PER_SALES_ORDER, cppu2.MAX_PER_SALES_ORDER) AS current_cart_rule
FROM COHORT_PRODUCT_PACKING_UNITS cppu 
JOIN PACKING_UNIT_PRODUCTS pup ON cppu.PRODUCT_PACKING_UNIT_ID = pup.id 
JOIN cohorts c ON c.id = cppu.cohort_id
LEFT JOIN COHORT_PRODUCT_PACKING_UNITS cppu2 
    ON cppu.PRODUCT_PACKING_UNIT_ID = cppu2.PRODUCT_PACKING_UNIT_ID 
    AND cppu2.cohort_id = c.FALLBACK_COHORT_ID
WHERE cppu.cohort_id IN ({",".join(map(str, COHORT_IDS))})
    AND pup.basic_unit_count = 1
'''

def get_current_cart_rules():
    """Get fresh cart rules."""
    print("Fetching current cart rules...")
    df = query_snowflake(CART_RULES_QUERY)
    df = df.groupby(['cohort_id', 'product_id']).agg(
        current_cart_rule=('current_cart_rule', 'min')
    ).reset_index()
    print(f"  Loaded {len(df)} records")
    return df


In [ ]:
# =============================================================================
# READY
# =============================================================================
print("\n" + "="*50)
print("QUERIES MODULE READY")
print("="*50)
print("Functions:")
print("  • get_current_stocks()")
print("  • get_current_prices()")
print("  • get_current_wac()")
print("  • get_current_cart_rules()")
print("\nNote: Market prices use MODULE_1_INPUT data")
